In [ ]:
import pickle
import gzip
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import time
import matplotlib
import math
import matplotlib.font_manager as fm
import matplotlib.ticker as mtick
font = fm.FontProperties()
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
def hide_spines(intx=False,inty=False):
    """Hides the top and rightmost axis spines from view for all active
    figures and their respective axes."""

    # Retrieve a list of all current figures.
    figures = [x for x in matplotlib._pylab_helpers.Gcf.get_all_fig_managers()]
    if (plt.gca().get_legend()):
        plt.setp(plt.gca().get_legend().get_texts(), fontproperties=font) 
    for figure in figures:
        # Get all Axis instances related to the figure.
        for ax in figure.canvas.figure.get_axes():
            # Disable spines.
            ax.spines['right'].set_color('none')
            ax.spines['top'].set_color('none')
            # Disable ticks.
            ax.xaxis.set_ticks_position('bottom')
            ax.yaxis.set_ticks_position('left')
           # ax.xaxis.set_major_formatter(mtick.FuncFormatter(lambda v,_: ("10$^{%d}$" % math.log(v,10)) ))
            for label in ax.get_xticklabels() :
                label.set_fontproperties(font)
            for label in ax.get_yticklabels() :
                label.set_fontproperties(font)
            #ax.set_xticklabels(ax.get_xticks(), fontproperties = font)
            ax.set_xlabel(ax.get_xlabel(), fontproperties = font)
            ax.set_ylabel(ax.get_ylabel(), fontproperties = font)
            ax.set_title(ax.get_title(), fontproperties = font)
            if (inty):
                ax.yaxis.set_major_formatter(mtick.FormatStrFormatter('%d'))
            if (intx):
                ax.xaxis.set_major_formatter(mtick.FormatStrFormatter('%d'))
def show(nm,a=0,b=0):
    hide_spines(a,b)
    #ax.xaxis.set_major_formatter(mtick.FuncFormatter(lambda v,_: ("10$^{%d}$" % math.log(v,10)) ))
    #plt.yticks([1,1e-2,1e-4,1e-6,1e-8,1e-10,1e-12], labels)
    #ax.yaxis.set_major_formatter(mtick.FuncFormatter(lambda v,_: ("10$^{%d}$" % math.log(v,10)) ))
    plt.savefig(nm);
    plt.show()
plt.rcParams['axes.titlesize'] = 18
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['xtick.labelsize'] = 18
plt.rcParams['ytick.labelsize'] = 18
#this line is only needed in iPython notebooks
%matplotlib inline
pickle_off = open("lopex_spectra.pkl","rb")
unpckl = pickle.Unpickler(pickle_off,encoding='latin1')
db = unpckl.load()

In [ ]:
#let's build a big matrix
leaf_ref = np.zeros((330,2101))
for i in range(1,331):
    fname = "00" + str(i)
    fname = fname[len(str(i))-1:len(fname)]
    tmp = db["lo93r0" + fname + ".txt"]
    leaf_ref[i-1,:] = tmp[:,1]
wlngth = tmp[:,0]
print(wlngth.shape)
plt.plot(leaf_ref[0,:], label="1")
plt.plot(leaf_ref[142,:], label="2")

In [ ]:
U, s, V = np.linalg.svd(leaf_ref, full_matrices=False)
print(U.shape, s.shape,V.shape)
plt.semilogy(s)
plt.show()
plt.plot(np.cumsum(s[0:10]**2)/np.sum(s**2))
print(np.cumsum(s[0:100]**2)/np.sum(s[0:100]**2))

In [ ]:
reduced_s = s*0
row = 10
for i in range(0,20,5):
    reduced_s[0:i] = s[0:i]
    S = np.diag(reduced_s)
    recon_data = np.dot(U, np.dot(S, V))
    plt.plot(recon_data[row,50:250], label=str(i+1))
plt.plot(leaf_ref[row,50:250],label="Original")
plt.legend(loc="best")
plt.show()

In [ ]:
print(np.argmax(U[:,1]),np.argmin(U[:,1]))
plt.plot(V[1,:])
plt.show()
plt.plot(leaf_ref[np.argmax(U[:,1]),:],label="max U1 " + str(np.argmax(U[:,1])))
plt.plot(leaf_ref[np.argmin(U[:,1]),:],label="min U1 " + str(np.argmin(U[:,1])))
plt.legend(loc="best")

In [ ]:
print(np.argmax(U[:,2]),np.argmin(U[:,2]))
plt.plot(V[2,:])
plt.show()
plt.plot(leaf_ref[np.argmax(U[:,2]),:],label="max U2 " + str(np.argmax(U[:,2])))
plt.plot(leaf_ref[np.argmin(U[:,2]),:],label="min U2 " + str(np.argmin(U[:,2])))
plt.legend(loc="best")

In [ ]:
def xyz_from_xy(x, y):
    """Return the vector (x, y, 1-x-y)."""
    return np.array((x, y, 1-x-y))

class ColourSystem:
    """A class representing a colour system.

    A colour system defined by the CIE x, y and z=1-x-y coordinates of
    its three primary illuminants and its "white point".

    TODO: Implement gamma correction

    """

    # The CIE colour matching function for 380 - 780 nm in 5 nm intervals
    cmf = np.loadtxt('cie-cmf.txt', usecols=(1,2,3))

    def __init__(self, red, green, blue, white):
        """Initialise the ColourSystem object.

        Pass vectors (ie NumPy arrays of shape (3,)) for each of the
        red, green, blue  chromaticities and the white illuminant
        defining the colour system.

        """

        # Chromaticities
        self.red, self.green, self.blue = red, green, blue
        self.white = white
        # The chromaticity matrix (rgb -> xyz) and its inverse
        self.M = np.vstack((self.red, self.green, self.blue)).T 
        self.MI = np.linalg.inv(self.M)
        # White scaling array
        self.wscale = self.MI.dot(self.white)
        # xyz -> rgb transformation matrix
        self.T = self.MI / self.wscale[:, np.newaxis]

    def xyz_to_rgb(self, xyz, out_fmt=None):
        """Transform from xyz to rgb representation of colour.

        The output rgb components are normalized on their maximum
        value. If xyz is out the rgb gamut, it is desaturated until it
        comes into gamut.

        By default, fractional rgb components are returned; if
        out_fmt='html', the HTML hex string '#rrggbb' is returned.

        """

        rgb = self.T.dot(xyz)
        if np.any(rgb < 0):
            # We're not in the RGB gamut: approximate by desaturating
            w = - np.min(rgb)
            rgb += w
        if not np.all(rgb==0):
            # Normalize the rgb vector
            rgb /= np.max(rgb)

        if out_fmt == 'html':
            return self.rgb_to_hex(rgb)
        return rgb

    def rgb_to_hex(self, rgb):
        """Convert from fractional rgb values to HTML-style hex string."""

        hex_rgb = (255 * rgb).astype(int)
        return '#{:02x}{:02x}{:02x}'.format(*hex_rgb)

    def spec_to_xyz(self, spec):
        """Convert a spectrum to an xyz point.

        The spectrum must be on the same grid of points as the colour-matching
        function, self.cmf: 380-780 nm in 5 nm steps.

        """

        XYZ = np.sum(spec[:, np.newaxis] * self.cmf, axis=0)
        den = np.sum(XYZ)
        if den == 0.:
            return XYZ
        return XYZ / den

    def spec_to_rgb(self, spec, out_fmt=None):
        """Convert a spectrum to an rgb value."""

        xyz = self.spec_to_xyz(spec)
        return self.xyz_to_rgb(xyz, out_fmt)

illuminant_D65 = xyz_from_xy(0.3127, 0.3291)
cs_hdtv = ColourSystem(red=xyz_from_xy(0.67, 0.33),
                       green=xyz_from_xy(0.21, 0.71),
                       blue=xyz_from_xy(0.15, 0.06),
                       white=illuminant_D65)

cs_smpte = ColourSystem(red=xyz_from_xy(0.63, 0.34),
                        green=xyz_from_xy(0.31, 0.595),
                        blue=xyz_from_xy(0.155, 0.070),
                        white=illuminant_D65)

cs_srgb = ColourSystem(red=xyz_from_xy(0.64, 0.33),
                       green=xyz_from_xy(0.30, 0.60),
                       blue=xyz_from_xy(0.15, 0.06),
                       white=illuminant_D65)

In [ ]:
from scipy.constants import h, c, k
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from matplotlib.patches import Wedge

cs = cs_hdtv

def planck(lam, T):
    """ Returns the spectral radiance of a black body at temperature T.

    Returns the spectral radiance, B(lam, T), in W.sr-1.m-2 of a black body
    at temperature T (in K) at a wavelength lam (in nm), using Planck's law.

    """

    lam_m = lam / 1.e9
    fac = h*c/lam_m/k/T
    B = 2*h*c**2/lam_m**5 / (np.exp(fac) - 1)
    return B

fig, ax = plt.subplots()

# The grid of visible wavelengths corresponding to the grid of colour-matching
# functions used by the ColourSystem instance.
lam = np.arange(380., 781., 5)

for i in range(24):
    # T = 500 to 12000 K
    T = 500*i + 500

    # Calculate the black body spectrum and the HTML hex RGB colour string
    # it looks like
    spec = planck(lam, T)
    html_rgb = cs.spec_to_rgb(spec, out_fmt='html')

    # Place and label a circle with the colour of a black body at temperature T
    x, y = i % 6, -(i // 6)
    circle = Circle(xy=(x, y*1.2), radius=0.4, fc=html_rgb)
    ax.add_patch(circle)
    ax.annotate('{:4d} K'.format(T), xy=(x, y*1.2-0.5), va='center',
                ha='center', color=html_rgb)

# Set the limits and background colour; remove the ticks
ax.set_xlim(-0.5,5.5)
ax.set_ylim(-4.35, 0.5)
ax.set_xticks([])
ax.set_yticks([])
ax.set_facecolor('k')
# Make sure our circles are circular!
ax.set_aspect("equal")
plt.show()
plt.plot(lam,planck(lam, 5800))

In [ ]:
from scipy import interpolate
lnames=np.loadtxt("leafnames.csv",dtype=bytes, delimiter="\n").astype(str)
numcol = 8
fig, axarr = plt.subplots(numcol,numcol ,figsize=(12,12),facecolor="k")
illuminant_D65 = np.array([0.941,1,1.098])/np.sum(np.array([0.941,1,1.098]))
rdtri = np.array([.506,.237,0])
rdtri /= np.sum(rdtri)
grtri = np.array([.304,.707,.042])
grtri /= np.sum(grtri)
btri = np.array([.154,.056,.784])
btri /= np.sum(btri)
cs = ColourSystem(red=rdtri,
                       green=grtri,
                       blue=btri,
                       white=illuminant_D65)

reduced_s = s*0
reduced_s[0:2] = s[0:2]
S = np.diag(reduced_s)
recon_data2 = np.dot(U, np.dot(S, V))
reduced_s = s*0

reduced_s[0:5] = s[0:5]
S = np.diag(reduced_s)
recon_data5 = np.dot(U, np.dot(S, V))
for i in range(numcol**2):
    leaf = i*5
    lam = np.arange(380., 781., 5)
    resamp = interpolate.interp1d(wlngth, leaf_ref[leaf,:], fill_value="extrapolate")
    html_rgb = cs.spec_to_rgb(resamp(np.arange(380., 781., 5))*planck(lam, 5800), out_fmt='html')
    resamp_red = interpolate.interp1d(wlngth, recon_data2[leaf,:], fill_value="extrapolate")
    html_rgb_red = cs.spec_to_rgb(resamp_red(np.arange(380., 781., 5))*planck(lam, 5800), out_fmt='html')
    resamp_red5 = interpolate.interp1d(wlngth, recon_data5[leaf,:], fill_value="extrapolate")
    html_rgb_red5 = cs.spec_to_rgb(resamp_red5(np.arange(380., 781., 5))*planck(lam, 5800), out_fmt='html')
    # Place and label a circle with the colour of a black body at temperature T
    x, y = (i % numcol, i//numcol)
    rad = 4
    circle = Wedge(center=(0,0), r=rad, theta1=0, theta2=np.pi*360/(2*np.pi), fc=html_rgb)
    axarr[x,y].add_patch(circle)
    circle = Wedge(center=(0,0), r=rad, theta1=180, theta2=270, fc=html_rgb_red)
    axarr[x,y].add_patch(circle)
    circle = Wedge(center=(0,0), r=rad, theta1=270, theta2=360, fc=html_rgb_red5)
    axarr[x,y].add_patch(circle)
    axarr[x,y].set_xlim(-rad-.5,rad+.5)
    axarr[x,y].set_ylim(-rad-3,rad+.5)
    axarr[x,y].set_facecolor('k')
    axarr[x,y].set_aspect("equal")
    axarr[x,y].set_xticks([])
    axarr[x,y].set_yticks([])
    axarr[x,y].annotate(lnames[leaf], xy=(0,-rad-1), va='center',
                ha='center', color=html_rgb)
plt.tight_layout()
plt.savefig("colors.jpg")
plt.show()

In [ ]:

from time import time
from sklearn.manifold import (TSNE)
from sklearn import datasets
from sklearn.decomposition import PCA
pca = PCA(n_components=50)
xmake = pca.fit_transform(leaf_ref) 

tsne= TSNE(n_components=2, init='pca',perplexity=40, random_state=0,learning_rate=1000.0,n_iter=5000,n_iter_without_progress=1000)
t0 = time()
X_tsne = tsne.fit_transform(leaf_ref)

In [ ]:

from matplotlib import offsetbox
X = X_tsne.copy()
x_min, x_max = np.min(X, 0), np.max(X, 0)
X = (X - x_min) / (x_max - x_min)
plt.figure(figsize=(5,5))
ax = plt.subplot(111)
for i in range(X.shape[0]):
    lam = np.arange(380., 781., 5)
    resamp = interpolate.interp1d(wlngth, leaf_ref[i,:], fill_value="extrapolate")
    html_rgb = cs.spec_to_rgb(resamp(np.arange(380., 781., 5))*planck(lam, 5800), out_fmt='html')
    plt.scatter(X[i, 0], X[i, 1],c=html_rgb)
plt.xticks([]), plt.yticks([])
plt.show()



In [ ]:

from matplotlib.offsetbox import (TextArea, DrawingArea, OffsetImage,
                                  AnnotationBbox)
from matplotlib.patches import Circle

plt.figure(figsize=(17,22))
ax = plt.subplot(111)
shown_images = np.array([[1., 1.]])  # just something big
for i in range(X.shape[0]-1):
    dist = np.sum((X[i,:] - shown_images) ** 2, 1)
    if np.min(dist) < 1e-4:
        # don't show points that are too close
        continue
    lam = np.arange(380., 781., 5)
    shown_images = np.r_[shown_images, [X[i]]]
    resamp = interpolate.interp1d(wlngth, leaf_ref[i,:], fill_value="extrapolate")
    html_rgb = cs.spec_to_rgb(resamp(np.arange(380., 781., 5))*planck(lam, 5800), out_fmt='html')
    da = DrawingArea(40, 40, 0, 0)
    p = Circle((10, 10), 10,color=html_rgb)
    offsetbox = TextArea(lnames[i])
    xy = X[i]
    ab = AnnotationBbox(offsetbox, xy,
                        xybox=(xy[0], xy[1]),
                        xycoords='data',
                        boxcoords=("axes fraction", "data"),
                        box_alignment=(0., 0.5),
                        arrowprops=dict(arrowstyle="->"))
    ax.add_artist(ab)
plt.xticks([]), plt.yticks([])

In [ ]:
pickle_off = open("lopex_5d.pkl","rb")
unpckl = pickle.Unpickler(pickle_off,encoding='latin1')
db = unpckl.load()
leaf_ref_ae = np.zeros((330,2101))
for i in range(1,331):
    fname = "00" + str(i)
    fname = fname[len(str(i))-1:len(fname)]
    tmp = db["lo93r0" + fname + ".txt"]
    leaf_ref_ae[i-1,:] = tmp[:,2]
wlngth = tmp[:,0]
plt.plot(leaf_ref[142,:], label="1")
plt.plot(leaf_ref_ae[142,:], label="2")

In [ ]:
from scipy import interpolate
lnames=np.loadtxt("leafnames.csv",dtype=bytes, delimiter="\n").astype(str)
numcol = 8
fig, axarr = plt.subplots(numcol,numcol ,figsize=(20,20),facecolor="k")
illuminant_D65 = np.array([0.941,1,1.098])/np.sum(np.array([0.941,1,1.098]))
rdtri = np.array([.506,.237,0])
rdtri /= np.sum(rdtri)
grtri = np.array([.304,.707,.042])
grtri /= np.sum(grtri)
btri = np.array([.154,.056,.784])
btri /= np.sum(btri)
cs = ColourSystem(red=rdtri,
                       green=grtri,
                       blue=btri,
                       white=illuminant_D65)

reduced_s = s*0
for j in range(2):
    reduced_s[j] = s[j]
    S = np.diag(reduced_s)
    recon_data2 = np.dot(U, np.dot(S, V))
reduced_s = s*0
for j in range(5):
    reduced_s[j] = s[j]
    S = np.diag(reduced_s)
    recon_data5 = np.dot(U, np.dot(S, V))
for i in range(numcol**2):
    leaf = i*5
    lam = np.arange(380., 781., 5)
    resamp = interpolate.interp1d(wlngth, leaf_ref[leaf,:], fill_value="extrapolate")
    html_rgb = cs.spec_to_rgb(resamp(np.arange(380., 781., 5))*planck(lam, 5800), out_fmt='html')
    resamp_red = interpolate.interp1d(wlngth, leaf_ref_ae[leaf,:], fill_value="extrapolate")
    html_rgb_red = cs.spec_to_rgb(resamp_red(np.arange(380., 781., 5))*planck(lam, 5800), out_fmt='html')
    resamp_red5 = interpolate.interp1d(wlngth, recon_data5[leaf,:], fill_value="extrapolate")
    html_rgb_red5 = cs.spec_to_rgb(resamp_red5(np.arange(380., 781., 5))*planck(lam, 5800), out_fmt='html')
    # Place and label a circle with the colour of a black body at temperature T
    x, y = (i % numcol, i//numcol)
    rad = 4
    circle = Wedge(center=(0,0), r=rad, theta1=0, theta2=np.pi*360/(2*np.pi), fc=html_rgb)
    axarr[x,y].add_patch(circle)
    circle = Wedge(center=(0,0), r=rad, theta1=180, theta2=270, fc=html_rgb_red)
    axarr[x,y].add_patch(circle)
    circle = Wedge(center=(0,0), r=rad, theta1=270, theta2=360, fc=html_rgb_red5)
    axarr[x,y].add_patch(circle)
    axarr[x,y].set_xlim(-rad-.5,rad+.5)
    axarr[x,y].set_ylim(-rad-3,rad+.5)
    axarr[x,y].set_facecolor('k')
    axarr[x,y].set_aspect("equal")
    axarr[x,y].set_xticks([])
    axarr[x,y].set_yticks([])
    axarr[x,y].annotate(lnames[leaf], xy=(0,-rad-1), va='center',
                ha='center', color=html_rgb)
plt.tight_layout()
plt.savefig("colors_5dcomp.pdf")
plt.show()

In [ ]:
pickle_off = open("lopex_2d.pkl","rb")
unpckl = pickle.Unpickler(pickle_off,encoding='latin1')
db = unpckl.load()
leaf_ref_ae2 = np.zeros((330,2101))
for i in range(1,331):
    fname = "00" + str(i)
    fname = fname[len(str(i))-1:len(fname)]
    tmp = db["lo93r0" + fname + ".txt"]
    leaf_ref_ae2[i-1,:] = tmp[:,2]
wlngth = tmp[:,0]
plt.plot(leaf_ref[142,:], label="1")
plt.plot(leaf_ref_ae[142,:], label="5")
plt.plot(leaf_ref_ae2[142,:], label="2")
plt.legend()

In [ ]:
from scipy import interpolate
lnames=np.loadtxt("leafnames.csv",dtype=bytes, delimiter="\n").astype(str)
solar = np.loadtxt("solar.csv",delimiter=",")
solar_interp = interpolate.interp1d(solar[:,0], solar[:,3])
plt.plot(np.arange(380., 781., 5),solar_interp(np.arange(380., 781., 5))*np.mean(planck(lam,5800))/np.mean(solar_interp(np.arange(380., 781., 5))))
plt.plot(np.arange(380., 781., 5),planck(lam, 5800))
plt.show()
numcol = 4
fig, axarr = plt.subplots(numcol,numcol ,figsize=(20,15),facecolor="k")
illuminant_D65 = np.array([0.941,1,1.098])/np.sum(np.array([0.941,1,1.098]))
rdtri = np.array([.506,.237,0])
rdtri /= np.sum(rdtri)
grtri = np.array([.304,.707,.042])
grtri /= np.sum(grtri)
btri = np.array([.154,.056,.784])
btri /= np.sum(btri)
cs = ColourSystem(red=rdtri,
                       green=grtri,
                       blue=btri,
                       white=illuminant_D65)

reduced_s = s*0
for j in range(2):
    reduced_s[j] = s[j]
    S = np.diag(reduced_s)
    recon_data2 = np.dot(U, np.dot(S, V))
reduced_s = s*0
for j in range(5):
    reduced_s[j] = s[j]
    S = np.diag(reduced_s)
    recon_data5 = np.dot(U, np.dot(S, V))
for i in range(numcol**2):
    leaf = i*5*3
    lam = np.arange(380., 781., 5)
    resamp = interpolate.interp1d(wlngth, leaf_ref[leaf,:], fill_value="extrapolate")
    html_rgb = cs.spec_to_rgb(resamp(np.arange(380., 781., 5))*solar_interp(np.arange(380., 781., 5)), out_fmt='html')
    resamp_red = interpolate.interp1d(wlngth, recon_data2[leaf,:], fill_value="extrapolate")
    html_rgb_red = cs.spec_to_rgb(resamp_red(np.arange(380., 781., 5))*solar_interp(np.arange(380., 781., 5)), out_fmt='html')
    resamp_red5 = interpolate.interp1d(wlngth, recon_data5[leaf,:], fill_value="extrapolate")
    html_rgb_red5 = cs.spec_to_rgb(resamp_red5(np.arange(380., 781., 5))*solar_interp(np.arange(380., 781., 5)), out_fmt='html')
    resamp_red = interpolate.interp1d(wlngth, leaf_ref_ae2[leaf,:], fill_value="extrapolate")
    html_rgb_redae = cs.spec_to_rgb(resamp_red(np.arange(380., 781., 5))*solar_interp(np.arange(380., 781., 5)), out_fmt='html')
    resamp_red5 = interpolate.interp1d(wlngth, leaf_ref_ae[leaf,:], fill_value="extrapolate")
    html_rgb_redae5 = cs.spec_to_rgb(resamp_red5(np.arange(380., 781., 5))*solar_interp(np.arange(380., 781., 5)), out_fmt='html')
    # Place and label a circle with the colour of a black body at temperature T
    x, y = (i % numcol, i//numcol)
    rad = 1
    circle = Wedge(center=(0,0), r=rad, theta1=0, theta2=np.pi*360/(2*np.pi), fc=html_rgb)
    axarr[x,y].add_patch(circle)
    circle = Wedge(center=(0,0), r=rad, theta1=180, theta2=225, fc=html_rgb_red)
    axarr[x,y].add_patch(circle)
    circle = Wedge(center=(0,0), r=rad, theta1=225, theta2=270, fc=html_rgb_red5)
    axarr[x,y].add_patch(circle)
    circle = Wedge(center=(0,0), r=rad, theta1=270, theta2=315, fc=html_rgb_redae)
    axarr[x,y].add_patch(circle)
    circle = Wedge(center=(0,0), r=rad, theta1=315, theta2=360, fc=html_rgb_redae5)
    if (i == 0):
        axarr[x,y].annotate("Exact",xy=(0,0.5*rad), va='center',
                ha='center',size=20)
        axarr[x,y].annotate("2 PC",xy=(-0.5*rad,-0.25*rad), va='center',
                ha='center', rotation=35,size=15)
        axarr[x,y].annotate("5 PC",xy=(-0.2*rad,-0.5*rad), va='center',
                ha='center', rotation=70,size=15)
        axarr[x,y].annotate("2 AE",xy=(0.2*rad,-0.5*rad), va='center',
                ha='center', rotation=-70,size=15)
        axarr[x,y].annotate("5 AE",xy=(0.5*rad,-0.25*rad), va='center',
                ha='center', rotation=-35,size=15)
    
    axarr[x,y].add_patch(circle)
    axarr[x,y].set_xlim(-rad-.5,rad+.5)
    axarr[x,y].set_ylim(-rad-.95,rad+.5)
    axarr[x,y].set_facecolor('k')
    axarr[x,y].set_aspect("equal")
    axarr[x,y].set_xticks([])
    axarr[x,y].set_yticks([])
    axarr[x,y].annotate(lnames[leaf], xy=(0,-rad-.75), va='center',
                ha='center', color=html_rgb,size=30)
plt.tight_layout()
plt.savefig("colors_aecomp.pdf")
plt.show()

In [ ]:
print(np.argmax(U[:,1]),np.argmin(U[:,1]))
#plt.plot(V[1,:])
#plt.show()
plt.plot(wlngth,leaf_ref[np.argmax(U[:,1]),:],'k',label=lnames[np.argmax(U[:,1])]) #label="max U1 " + str(np.argmax(U[:,1])))
plt.plot(wlngth,leaf_ref[np.argmin(U[:,1]),:],'r',label=lnames[np.argmin(U[:,1])]) #label="max U1 " + str(np.argmax(U[:,1])))

plt.plot(wlngth,recon_data5[np.argmax(U[:,1]),:],'b--',label="5 term PC") #label="max U1 " + str(np.argmax(U[:,1])))
plt.plot(wlngth,leaf_ref_ae2[np.argmax(U[:,1]),:],'g:',label="2 term AE") #label="max U1 " + str(np.argmax(U[:,1])))
plt.plot(wlngth,recon_data5[np.argmin(U[:,1]),:],'b--') #label="max U1 " + str(np.argmax(U[:,1])))
plt.plot(wlngth,leaf_ref_ae2[np.argmin(U[:,1]),:],'g:') #label="max U1 " + str(np.argmax(U[:,1])))


plt.legend(loc="best")
plt.xlabel("$\lambda$ (nm)")
plt.ylabel("reflection coeff.")
show("Korn.pdf")
plt.plot(wlngth,recon_data5[np.argmin(U[:,1]),:],'--',label="5 term PC") #label="max U1 " + str(np.argmax(U[:,1])))
plt.plot(wlngth,leaf_ref_ae2[np.argmin(U[:,1]),:],':',label="2 term AE") #label="max U1 " + str(np.argmax(U[:,1])))
plt.legend(loc="best")
plt.xlabel("$\lambda$ (nm)")
plt.ylabel("reflection coeff.")
show("Iris.pdf")

plt.plot(leaf_ref[np.argmin(U[:,1]),:],label="min U1 " + str(np.argmin(U[:,1])))
plt.legend(loc="best")

In [ ]:
print(np.max(np.amax(np.abs(leaf_ref_ae2 - leaf_ref)/leaf_ref,axis=1)))
print(np.max(np.amax(np.abs(recon_data5 - leaf_ref)/leaf_ref,axis=1)))

In [ ]:
print(np.mean(np.amax(np.abs(leaf_ref_ae2 - leaf_ref)/leaf_ref,axis=1)))
print(np.mean(np.amax(np.abs(recon_data5 - leaf_ref)/leaf_ref,axis=1)))

In [ ]:
print(np.argmax(np.max(np.abs(leaf_ref_ae2 - leaf_ref)/leaf_ref,axis=1)))
print(np.argmax(np.max(np.abs(recon_data5 - leaf_ref)/leaf_ref,axis=1)))

In [ ]:
print(np.argmax(U[:,1]),np.argmin(U[:,1]))
#plt.plot(V[1,:])
#plt.show()
leaf1 = 48
leaf2 = 270
plt.plot(wlngth,leaf_ref[leaf1,:],'k',label=lnames[leaf1]) #label="max U1 " + str(np.argmax(U[:,1])))
plt.plot(wlngth,leaf_ref[leaf2,:],'r',label=lnames[leaf2]) #label="max U1 " + str(np.argmax(U[:,1])))

plt.plot(wlngth,recon_data5[leaf1,:],'b--',label="5 term PC") #label="max U1 " + str(np.argmax(U[:,1])))
plt.plot(wlngth,leaf_ref_ae2[leaf1,:],'g:',label="2 term AE") #label="max U1 " + str(np.argmax(U[:,1])))
plt.plot(wlngth,recon_data5[leaf2,:],'b--') #label="max U1 " + str(np.argmax(U[:,1])))
plt.plot(wlngth,leaf_ref_ae2[leaf2,:],'g:') #label="max U1 " + str(np.argmax(U[:,1])))


plt.legend(loc="best")
plt.xlabel("$\lambda$ (nm)")
plt.ylabel("reflection coeff.")
show("Korn_worst.pdf")
plt.plot(wlngth,recon_data5[np.argmin(U[:,1]),:],'--',label="5 term PC") #label="max U1 " + str(np.argmax(U[:,1])))
plt.plot(wlngth,leaf_ref_ae2[np.argmin(U[:,1]),:],':',label="2 term AE") #label="max U1 " + str(np.argmax(U[:,1])))
plt.legend(loc="best")
plt.xlabel("$\lambda$ (nm)")
plt.ylabel("reflection coeff.")
show("Iris.pdf")

plt.plot(leaf_ref[np.argmin(U[:,1]),:],label="min U1 " + str(np.argmin(U[:,1])))
plt.legend(loc="best")